# Processing and Analysis

## Using Existing Tools

To call a GRASS tool from Python, we will use _run_command_ from the _grass.script_ package (typically imported as _gs_):

```python
gs.run_command(
    "r.viewshed",
    input=elevation,
    output="viewshed",
    coordinates=(1200, 3000)
)
```

For tools with textual outputs, _read_command_ can be used to capture that textual output in Python (as a string):

```python
table_data = gs.read_command(
    "r.univar",
    map="elevation",
    flags="t",
    separator="comma",
)
```

There are other ways of calling GRASS tools from Python, but using functions from the _run_command_ family is the most typical way for GRASS tools.

## Processing Parameters

The values parsed from the command line are stored in a dictionary returned by the _parse_ function. They can be accessed using `dictionary["name"]` syntax where _name_ is the name of the parameter.

In [ ]:
%%writefile viewscape.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: raster
# % keyword: statistics
# % keyword: viewshed
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# %end
# %option G_OPT_M_COORDS
# %end
# %option G_OPT_R_INPUTS
# %end
# %option G_OPT_F_OUTPUT
# %end

import subprocess
import sys
import csv
import io

import grass.script as gs


def viewshed(
    elevation,
    coords,
    output,
    sample_continuous,
):
    name = "viewshed"
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=name,
        coordinates=coords,
        flags="cb",
    )
    gs.run_command("r.null", map=name, setnull=0)
    results = {}
    results["x"] = coords[0]
    results["y"] = coords[1]
    for raster in sample_continuous:
        table_data = gs.read_command(
            "r.univar",
            map=raster,
            zones=name,
            quiet=True,
            flags="t",
            separator="comma",
        )
        # While we could use .strip().splitlines()[-1].split(",") here,
        # using a proper CSV reader is more robust.
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            for key, value in row.items():
                new_key = f"{raster}_{key}"
                results[new_key] = value
    print(results)


def main():
    options, flags = gs.parser()
    coords = options["coordinates"].split(",")
    sample_continuous = options["input"].split(",")
    viewshed(
        elevation=options["input"],
        # We will be storing the coordinate values, so we convert them to floats
        # instead of just passing them to r.viewshed as a string (string would
        # work just fine for that).
        coords=(float(coords[0]), float(coords[1])),
        output=options["output"],
        sample_continuous=sample_continuous,
    )


if __name__ == "__main__":
    main()

As before, we will make the script executable:

In [ ]:
!chmod u+x viewscape.py
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec g.region raster=elevation

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation coordinates=639889.984967892,226698.08042942305 output="data.txt" input=elevation --o

## View the Data

To view the data in the notebook we will render the raster using _grass.jupyter.Map_. Usually, we would just create a GRASS session in the notebook. However, to keep our development environment as is, we will avoid creating a session in the notebook process, but we will use a subprocess to do the rendering into a PNG image. The following uses the `%%python` magic to execute Python code in a subprocess:

In [ ]:
%%python
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.jupyter as gj
import grass.script.setup  # Needed only in 8.2 and older.

with grass.script.setup.init("~/grassdata/nc_spm_08_grass7/foss4g") as session:
    gs.run_command("r.colors", map="viewshed", color="grey")
    ortho_map = gj.Map(use_region=True)
    ortho_map.d_rast(map="elevation")
    ortho_map.d_rast(map="viewshed")
    # Save the image (in a standard notebook, we would just display the image now).
    ortho_map.save("result.png")

Now, use _Image_ from _IPython.display_ to display the PNG:

In [ ]:
from IPython.display import Image

Image("result.png")

## Using r.univar

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec r.univar map=elevation zones=viewshed -t separator=comma

## Task: Implement JSON Output

```python
json.dump(...)
```

## Task: Run the Tool with Two Rasters for Sampling

The following computes NDVI so we have second continuous raster to sample:

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi

## Bonus Task: Implement CSV Output Instead of JSON

```python
csv.DictWriter(...)
```